In [1]:
#ссылка на тестовые данные
test_data_link = "https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv"

Большую часть времени работал над поиском по дампам википедии, но не успел:(

Предлагается в бейзлайне заменять вопросительное слово на ответ, что получить из вопросов утверждения c более коректной постановкой слова, что в теории должно помогать находить более корректные вероятности.


# Скачивание данных и библиотек

In [2]:
!pip install -U pip setuptools wheel
!pip install transformers
!pip install wget

import wget
import os
import zipfile

if not os.path.isfile('test_pipeline_questions.csv'):
  wget.download(test_data_link)

if not os.path.isfile('2021 VK Cup ML Finals.zip'):
  wget.download("https://www.dropbox.com/sh/ds8wh42kftccd6f/AAAOtEW26zOauBc7KOx-FY4ba?dl=1")
  with zipfile.ZipFile('2021 VK Cup ML Finals.zip', 'r') as zip_ref:
    zip_ref.extractall()

     |████████████████████████████████| 1.6 MB 31.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 2.6 MB 32.2 MB/s 
     |████████████████████████████████| 636 kB 44.7 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=3d83d0c65cb6b69515b8525f91af03434f93ede258497aa8c2b5c443f5ae1b18
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
import pandas as pd
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertForMaskedLM, RobertaForMaskedLM, RobertaTokenizer, RobertaForMaskedLM, RobertaTokenizer
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
tqdm.pandas()
# colab уперся в лимит, поэтому приходитться использовать cpu
GLOBAL_DEVICE = 'cpu'


# Код общего назначения

In [4]:
class Data:
  def __init__(self, test_size=0.25):
    self.train_mode = None
    self.train = None
    self.test = None
    self.test_size = test_size
  
  def __get_rus(self, seed):
    rus_data = pd.read_csv("rus_train_dataset.csv", delimiter='|')
    rus_data = rus_data.rename(columns={'Unnamed: 0': 'index'})
    if self.test_size != 1:
      rus_train, rus_test = train_test_split(rus_data, random_state=seed, test_size=self.test_size)
    else:
      rus_test = rus_data
      rus_train = rus_data.head(0)
    return rus_train, rus_test

  def __get_eng(self):
    eng_data = pd.read_csv("en_to_rus_train_dataset.csv", delimiter='|')
    eng_data = eng_data.rename(columns={'Unnamed: 0': 'index'}) 
    return eng_data

  def load_train_test(self, mode, seed):
    print(f'Данные загружаютсяб режим {mode}')
    rus_train, rus_test = self.__get_rus(seed)
    if mode == "RUS":
      self.train = rus_train
      self.test = rus_test

    elif mode == "ALL":
      eng_train = self.__get_eng()
      self.train = rus_train.append(eng_train, ignore_index=True)
      self.test = rus_test
    else:
      ValueError('Wrong value for mode')

      self.train['question'] = self.train['question'].replace('ё', 'е')
      self.test['question'] = self.test['question'].replace('ё', 'е')
    return None

  def get_train(self, mode='RUS', seed=42):
    if self.train_mode != mode:
      self.load_train_test(mode, seed)
    return self.train

  def get_test(self, mode='RUS', seed=42):
    if self.train_mode != mode:
      self.load_train_test(mode, seed)
    return self.test
  


def get_test_accuracy(data, model):
  test = data.get_test()
  my_prediction = model.predict(data)
  merged = test.merge(my_prediction, on='index')
  accuracy = (merged['right_answer_id'] == merged['my_prediction']).sum() / merged.shape[0]
  return merged, accuracy

def create_submission(questions_csv_path):
  model = BaselineWordPos()
  data = pd.read_csv('test_pipeline_questions.csv', encoding='utf-8', sep='|')
  answers = [str(model.predict_one(data['question'][i], 
                     [data['ps_0'][i], data['ps_1'][i], data['ps_2'][i]])) for i in range(len(data))]
  
  with open('answer.txt', 'w') as f:
    f.write('\n'.join(answers))
  return answers
  

In [ ]:
class BaselineWordPos:
  question_words = ['Кто',
                    'Кого',
                    'Кому',
                    'Кем',
                    'Что',
                    'Ком',
                    'Какой',
                    'Какого',
                    'Какому',
                    'Каким',
                    'Каком',
                    'Какую',
                    'Какая',
                    'Какой',
                    'Который',
                    'Которого',
                    'Которому',
                    'Которым',
                    'Где',
                    'Когда',
                    'Почему',
                    'Зачем',
                    'Куда',
                    'Откуда',
                    'Сколько',
                    'Чей',
                    'Как',
                    'чем',
                    'Какое',
                    'Какие',
                    'Скольким',
                    'Скольких',
                    'Столько',
                    'Стольких',
                    'Именно',
                    'чем',
                    'чего',
                    'насколько',
                    'каких',
                    'какова',
                    'Самый',
                    'Самая',
                    'Самое',
                    'Какими'
                    ]
  question_words_lower = set([word.lower() for word in question_words])

  def __init__(self):
    model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
    self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
    self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(GLOBAL_DEVICE)
  
  def calc_likelihood(self, text):

    tokens_ids = self.tokenizer.encode(text, return_tensors="pt").to(GLOBAL_DEVICE)
    prob = self.model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood


  def get_changed_question(self, question, answer):
    changed_question = question

    word_found = True

    for word in self.question_words:
      changed_question = " ".join([answer if word_from_question.lower() == word.lower() else word_from_question for word_from_question in changed_question.split(' ')])

    if changed_question == question:
      changed_question = question + ' ' + answer

      changed_question = " ".join([word for word in changed_question.split(" ") if word.lower() not in russian_stopwords])
    return changed_question

  def predict_one(self, question, answers):

    scores = [self.calc_likelihood(self.get_changed_question(question, answer)) for answer in answers]
    return np.argmax(scores)

  def predict(self, data):
    predict_data = data.get_test()
    prediction = predict_data.progress_apply(lambda x: self.predict_one(x['question'], [x['ps_0'], x['ps_1'], x['ps_2']]), axis=1)
    return pd.DataFrame({ 'index': predict_data['index'],
                          'my_prediction': prediction})


data = Data(test_size=1)
model = BaselineWordPos()

merged, accuracy = get_test_accuracy(data, model)
print(accuracy)